In [1]:
!rmdir /S /Q lightning_logs

In [2]:
import feature_engine.encoding
import feature_engine.pipeline
import feature_engine.selection
import feature_engine.wrappers
import lightning
import pandas
import sklearn.preprocessing
import torch
import torchmetrics

In [3]:
lightning.seed_everything(0)

Seed set to 0


0

In [4]:
train_df = pandas.read_csv("train.csv")
train_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900


In [5]:
train_dropped_df = train_df.dropna()
train_dropped_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,Chevrolet,Camaro Z28,1999,110000,Gasoline,310.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,Yes,14500
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900


In [6]:
train_X_df = train_dropped_df.drop(columns=["price"])
train_y_df = train_dropped_df[["price"]]

In [7]:
pipeline = feature_engine.pipeline.make_pipeline(
    feature_engine.selection.DropFeatures(features_to_drop=["id"]),   
    feature_engine.wrappers.SklearnTransformerWrapper(transformer=sklearn.preprocessing.MinMaxScaler(), variables=["model_year", "milage"]),
    feature_engine.encoding.OrdinalEncoder(encoding_method='arbitrary', variables=["brand", "model", "fuel_type", "engine", "transmission", "ext_col", "int_col", "accident", "clean_title"]),
)

In [8]:
train_X_transformed_df = pipeline.fit_transform(train_X_df)
train_X_transformed_df

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,0,0,0.66,0.525809,0,0,0,0,0,0,0
1,1,1,0.56,0.353544,0,1,0,1,1,1,0
2,2,2,0.56,0.337444,1,2,0,2,0,0,0
3,3,3,0.86,0.047913,0,3,1,3,2,0,0
4,4,4,0.94,0.018000,0,4,2,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
188527,2,286,0.50,0.271425,0,305,0,4,0,0,0
188528,8,11,0.86,0.120771,0,11,1,4,1,0,0
188529,4,862,0.88,0.070388,0,220,3,4,2,1,0
188530,4,832,0.94,0.033465,0,16,2,4,2,0,0


In [9]:
price_scaler = sklearn.preprocessing.MinMaxScaler()
train_y_transformed_array = price_scaler.fit_transform(train_y_df)
train_y_transformed_array

array([[0.00074524],
       [0.00101589],
       [0.00403105],
       ...,
       [0.00948483],
       [0.02875935],
       [0.00914439]])

In [10]:
train_joined_df = train_X_transformed_df.copy()
train_joined_df["price"] = train_y_transformed_array
train_joined_df

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,0,0.66,0.525809,0,0,0,0,0,0,0,0.000745
1,1,1,0.56,0.353544,0,1,0,1,1,1,0,0.001016
2,2,2,0.56,0.337444,1,2,0,2,0,0,0,0.004031
3,3,3,0.86,0.047913,0,3,1,3,2,0,0,0.014566
4,4,4,0.94,0.018000,0,4,2,3,1,0,0,0.032350
...,...,...,...,...,...,...,...,...,...,...,...,...
188527,2,286,0.50,0.271425,0,305,0,4,0,0,0,0.004234
188528,8,11,0.86,0.120771,0,11,1,4,1,0,0,0.008638
188529,4,862,0.88,0.070388,0,220,3,4,2,1,0,0.009485
188530,4,832,0.94,0.033465,0,16,2,4,2,0,0,0.028759


In [11]:
class MyModel(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = torch.nn.Sequential(
            torch.nn.Linear(11, 16), torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
        )

    def forward(self, x):
        return self._stack(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        loss = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        return loss
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        return loss

In [12]:
class MyData(lightning.LightningDataModule):
    def __init__(self):
        super().__init__()

    def prepare_data(self):
        train_X = torch.tensor(train_X_transformed_df.values, dtype=torch.float32)
        train_y = torch.tensor(train_y_transformed_array, dtype=torch.float32)
        full_data = torch.utils.data.TensorDataset(train_X, train_y)
        n_train = int(len(full_data) * 0.8)
        n_val = int(len(full_data) * 0.1)
        n_test = len(full_data) - n_train - n_val
        self._train_data, self._val_data, self._test_data = torch.utils.data.random_split(full_data, [n_train, n_val, n_test])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self._train_data, batch_size=256)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self._val_data, batch_size=256)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self._test_data)

In [13]:
trainer = lightning.Trainer(max_epochs=100, callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")])
model = MyModel()
loader = MyData()
trainer.fit(model, datamodule=loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | _stack | Sequential | 209    | train
----------------------------------------------
209       Trainable params
0         Non-trainable params
209       Total params
0.001     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

In [14]:
trainer.test(datamodule=loader)

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\hsmtk\repo\fantastic-octo-dollop\used-car\lightning_logs\version_0\checkpoints\epoch=13-step=7126.ckpt
Loaded model weights from the checkpoint at C:\Users\hsmtk\repo\fantastic-octo-dollop\used-car\lightning_logs\version_0\checkpoints\epoch=13-step=7126.ckpt
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11`

Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.0009710512822493911
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0009710512822493911}]

In [17]:
!taskkill /im tensorboard.exe /f
!del /q %TMP%\.tensorboard-info\*
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

SUCCESS: The process "tensorboard.exe" with PID 12380 has been terminated.


In [18]:
test_df = pandas.read_csv("test.csv")
test_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,2014,83315,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes
125686,314219,Audi,Q7 55 Prestige,2019,29336,Gasoline,3.0 Liter Turbo,Automatic,White,Black,None reported,NaN
125687,314220,Audi,A6 3.0T Premium Plus,2012,77634,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes
125688,314221,Audi,Q7 3.0T Premium,2012,112000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes


In [19]:
test_filled_df = test_df.copy()
test_filled_df["fuel_type"] = test_df["fuel_type"].fillna("Gasoline")
test_filled_df["accident"] = test_df["accident"].fillna("None reported")
test_filled_df["clean_title"] = test_df["clean_title"].fillna("Yes")
test_filled_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,Yes
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,Yes
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,2014,83315,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes
125686,314219,Audi,Q7 55 Prestige,2019,29336,Gasoline,3.0 Liter Turbo,Automatic,White,Black,None reported,Yes
125687,314220,Audi,A6 3.0T Premium Plus,2012,77634,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes
125688,314221,Audi,Q7 3.0T Premium,2012,112000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes


In [22]:
test_filled_df.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

In [23]:
tmp_df = test_filled_df.copy()
test_transformed_df = pipeline.transform(tmp_df)
test_transformed_df

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\feature_engine\encoding\base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) model, engine, ext_col.
  warnings.warn(


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,9,299.0,0.82,0.241788,0,29.0,5,4.0,1,0,0
1,9,16.0,0.92,0.022331,2,14.0,3,1.0,2,0,0
2,6,68.0,0.96,0.069205,0,62.0,4,4.0,10,0,0
3,5,5.0,0.84,0.151045,0,680.0,18,263.0,2,0,0
4,5,646.0,0.88,0.145468,0,5.0,0,7.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
125685,4,341.0,0.80,0.205520,0,149.0,2,3.0,2,0,0
125686,5,425.0,0.90,0.072205,0,311.0,18,4.0,2,0,0
125687,5,197.0,0.76,0.191489,0,6.0,0,3.0,2,0,0
125688,5,1135.0,0.76,0.276365,0,6.0,0,3.0,2,0,0


In [25]:
with torch.no_grad():
    predicted = model(torch.tensor(test_transformed_df.values, dtype=torch.float32))

In [27]:
price_scaler.inverse_transform(predicted)

array([[28856.54143   ],
       [74490.6841746 ],
       [51805.66917515],
       ...,
       [28617.23874236],
       [22268.67920542],
       [31929.46947194]])